<h5>Classificação de movimentos da mão utilizando sinais EMG e LDA</h5>

In [13]:
from data_caracteristics import DataCaracteristics as DC
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

Lendo e obtendo as características do dataset para todos os 16 canais

In [27]:
channels = np.arange(1,17)
w = 0.15
fs = 2048
overlap = 0.075 
for subject in tqdm(range(1,21)):
    dc = DC(path="dados/sinais_subject{0}.csv".format(subject))
    for session in [1,2]:
        for ch in channels:
            rms = []
            var = []
            zcr = []
            wl = []
            rotule = []
            column = 's'+str(session)+'_ch'+str(ch)
            dc.data[column] = dc.bandpass(fs,20,500,dc.data[column],10)
            dc.data[column] = dc.bandstop(fs,50,dc.data[column],30)
            for mov in dc.data.movement.unique():
                rotule.extend(np.repeat(mov, len(dc.rms(dc.data[dc.data.movement == mov][column],w,fs, overlap))))
                rms.extend(dc.rms(dc.data[dc.data.movement == mov][column],w,fs, overlap))
                var.extend(dc.var(dc.data[dc.data.movement == mov][column],w,fs, overlap))
                zcr.extend(dc.zcr(dc.data[dc.data.movement == mov][column].reset_index(),w,fs, overlap))
                wl.extend(dc.wl(dc.data[dc.data.movement == mov][column],w,fs, overlap))
            df_aux = pd.DataFrame({'rms_'+column:rms, 'var_'+column:var, 'zcr_'+column:zcr, 'wl_'+column:wl, 'rotule':rotule})
            dc.data_carac = pd.concat((dc.data_carac,df_aux),axis=1)
            dc.data_carac = dc.data_carac.T.drop_duplicates().T
    dc.data_carac.to_csv("dados/carac_subject{0}.csv".format(subject), sep=';', index = False)

100%|██████████| 20/20 [24:03<00:00, 72.20s/it]


Classe para classificação dos sinais

In [2]:
class DataClassification:
    def __init__(self, path):
        self.data = pd.read_csv(path,sep=';')
        
    
    def lda_train(self,columns):
        self.x = self.data[columns].values
        self.y = self.data['rotule'].values
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.x, self.y,  test_size=0.2)
        self.lda = LinearDiscriminantAnalysis(solver='eigen')
        self.lda.fit(self.X_train, self.y_train)

    def lda_predict_acc(self):
        y_pred = self.lda.predict(self.X_test)
        score = 100*accuracy_score(self.y_test, y_pred)
        return score
    
    def lda_cross_validation(self):
        scores = cross_val_score(self.lda, self.x, self.y, cv=5, scoring='accuracy')
        return np.mean(scores)*100, np.std(scores)*100

Classificação de todos os 16 canais

In [14]:
inicio = time.time()
channels = np.arange(1,17)
data = pd.DataFrame()
for session in [1,2]:
    sub = []
    acc = []
    crossval = []
    crossval_std = []
    for subject in tqdm(range(1,21)):
        classify = DataClassification(path="dados/carac_subject{0}.csv".format(subject))
        column = 's'+str(session)+'_ch'
        rms_ch = ['rms_'+column+str(i) for i in channels]
        var_ch = ['var_'+column+str(i) for i in channels]
        zcr_ch = ['zcr_'+column+str(i) for i in channels]
        columns = rms_ch+var_ch+zcr_ch
        classify.lda_train(columns)
        acurate = [classify.lda_predict_acc()]
        cval, std_cval = classify.lda_cross_validation()
        acc.extend(acurate)
        crossval.extend([cval])
        crossval_std.extend([std_cval])
        sub.extend(['Sujeito '+str(subject)])
    data['rotulo'] = sub
    data['acc_sessao'+str(session)] = acc
    data['crossval_sessao'+str(session)+'_media'] = crossval
    data['crossval_sessao'+str(session)+'_std'] = crossval_std
data['acc_media'] = (data['acc_sessao1']+data['acc_sessao2'])/2
data = data.append({'rotulo': 'Média', 'acc_sessao1':data.acc_sessao1.mean(), 'acc_sessao2':data.acc_sessao2.mean(),
'crossval_sessao1_media':data.crossval_sessao1_media.mean(), 'crossval_sessao2_media':data.crossval_sessao2_media.mean(),
'crossval_sessao1_std':data.crossval_sessao1_std.mean(), 'crossval_sessao2_std':data.crossval_sessao2_std.mean(), 'acc_media':data.acc_media.mean()
}, ignore_index = True)
data = data.round(2)
fim = time.time()
print("Tempo de execucao: "+str(fim-inicio))
#data.to_csv("dados/acc_score_16ch.csv", sep=';', index = False)

100%|██████████| 20/20 [00:00<00:00, 33.31it/s]

Tempo de execucao: 1.3177070617675781



C:\Users\Lázaro\AppData\Local\Temp\ipykernel_9464\231452885.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [5]:
data

,rotulo,acc_sessao1,crossval_sessao1_media,crossval_sessao1_std,acc_sessao2,crossval_sessao2_media,crossval_sessao2_std,acc_media
0,Sujeito 1,97.87,96.58,3.95,97.87,95.72,3.55,97.87
1,Sujeito 2,100.00,90.56,9.46,100.00,95.69,3.06,100.00
2,Sujeito 3,97.87,92.22,4.04,97.87,94.86,6.83,97.87
3,Sujeito 4,100.00,99.57,0.85,95.74,87.05,4.80,97.87
4,Sujeito 5,95.74,94.85,2.89,100.00,96.98,2.22,97.87
5,Sujeito 6,91.49,93.52,3.06,93.62,93.11,4.18,92.55
6,Sujeito 7,100.00,95.74,6.45,100.00,98.29,1.59,100.00
7,Sujeito 8,97.87,95.73,6.45,97.87,94.38,2.22,97.87
8,Sujeito 9,100.00,96.98,1.73,100.00,94.88,5.48,100.00
9,Sujeito 10,95.74,93.08,5.41,97.87,97.85,2.71,96.81


Classificação de 8 canais

In [15]:
inicio = time.time()
channels = [2,4,6,8,10,12,14,16]
data = pd.DataFrame()
for session in [1,2]:
    sub = []
    acc = []
    crossval = []
    crossval_std = []
    for subject in tqdm(range(1,21)):
        classify = DataClassification(path="dados/carac_subject{0}.csv".format(subject))
        column = 's'+str(session)+'_ch'
        rms_ch = ['rms_'+column+str(i) for i in channels]
        var_ch = ['var_'+column+str(i) for i in channels]
        zcr_ch = ['zcr_'+column+str(i) for i in channels]
        columns = rms_ch+var_ch+zcr_ch
        classify.lda_train(columns)
        acurate = [classify.lda_predict_acc()]
        cval, std_cval = classify.lda_cross_validation()
        acc.extend(acurate)
        crossval.extend([cval])
        crossval_std.extend([std_cval])
        sub.extend(['Sujeito '+str(subject)])
    data['rotulo'] = sub
    data['acc_sessao'+str(session)] = acc
    data['crossval_sessao'+str(session)+'_media'] = crossval
    data['crossval_sessao'+str(session)+'_std'] = crossval_std
data['acc_media'] = (data['acc_sessao1']+data['acc_sessao2'])/2
data = data.append({'rotulo': 'Média', 'acc_sessao1':data.acc_sessao1.mean(), 'acc_sessao2':data.acc_sessao2.mean(),
'crossval_sessao1_media':data.crossval_sessao1_media.mean(), 'crossval_sessao2_media':data.crossval_sessao2_media.mean(),
'crossval_sessao1_std':data.crossval_sessao1_std.mean(), 'crossval_sessao2_std':data.crossval_sessao2_std.mean(), 'acc_media':data.acc_media.mean()
}, ignore_index = True)
data = data.round(2)
fim = time.time()
print("Tempo de execucao: "+str(fim-inicio))
#data.to_csv("dados/acc_score_8ch.csv", sep=';', index = False)

100%|██████████| 20/20 [00:00<00:00, 48.90it/s]

Tempo de execucao: 0.874337911605835



C:\Users\Lázaro\AppData\Local\Temp\ipykernel_9464\1383986071.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [7]:
data

,rotulo,acc_sessao1,crossval_sessao1_media,crossval_sessao1_std,acc_sessao2,crossval_sessao2_media,crossval_sessao2_std,acc_media
0,Sujeito 1,100.00,94.88,6.10,95.74,96.55,2.22,97.87
1,Sujeito 2,95.74,92.29,8.97,97.87,97.43,2.10,96.81
2,Sujeito 3,93.62,86.65,4.16,100.00,93.98,5.29,96.81
3,Sujeito 4,100.00,99.15,1.04,95.74,85.79,2.86,97.87
4,Sujeito 5,97.87,96.57,2.17,97.87,97.42,1.61,97.87
5,Sujeito 6,95.74,93.53,4.27,80.85,86.24,6.09,88.30
6,Sujeito 7,97.87,95.73,5.38,97.87,97.01,3.94,97.87
7,Sujeito 8,93.62,91.88,7.40,97.87,90.49,4.69,95.74
8,Sujeito 9,100.00,96.13,3.15,97.87,94.01,4.72,98.94
9,Sujeito 10,95.74,90.09,2.92,100.00,96.55,2.22,97.87


Classificação de 4 canais

In [16]:
inicio = time.time()
channels = [4,8,12,14]
data = pd.DataFrame()
for session in [1,2]:
    sub = []
    acc = []
    crossval = []
    crossval_std = []
    for subject in tqdm(range(1,21)):
        classify = DataClassification(path="dados/carac_subject{0}.csv".format(subject))
        column = 's'+str(session)+'_ch'
        rms_ch = ['rms_'+column+str(i) for i in channels]
        var_ch = ['var_'+column+str(i) for i in channels]
        zcr_ch = ['zcr_'+column+str(i) for i in channels]
        columns = rms_ch+var_ch+zcr_ch
        classify.lda_train(columns)
        acurate = [classify.lda_predict_acc()]
        cval, std_cval = classify.lda_cross_validation()
        acc.extend(acurate)
        crossval.extend([cval])
        crossval_std.extend([std_cval])
        sub.extend(['Sujeito '+str(subject)])
    data['rotulo'] = sub
    data['acc_sessao'+str(session)] = acc
    data['crossval_sessao'+str(session)+'_media'] = crossval
    data['crossval_sessao'+str(session)+'_std'] = crossval_std
data['acc_media'] = (data['acc_sessao1']+data['acc_sessao2'])/2
data = data.append({'rotulo': 'Média', 'acc_sessao1':data.acc_sessao1.mean(), 'acc_sessao2':data.acc_sessao2.mean(),
'crossval_sessao1_media':data.crossval_sessao1_media.mean(), 'crossval_sessao2_media':data.crossval_sessao2_media.mean(),
'crossval_sessao1_std':data.crossval_sessao1_std.mean(), 'crossval_sessao2_std':data.crossval_sessao2_std.mean(), 'acc_media':data.acc_media.mean()
}, ignore_index = True)
data = data.round(2)
fim = time.time()
print("Tempo de execucao: "+str(fim-inicio))
#data.to_csv("dados/acc_score_4ch.csv", sep=';', index = False)

100%|██████████| 20/20 [00:00<00:00, 49.95it/s]
C:\Users\Lázaro\AppData\Local\Temp\ipykernel_9464\3903794724.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Tempo de execucao: 0.8697383403778076


In [9]:
data

,rotulo,acc_sessao1,crossval_sessao1_media,crossval_sessao1_std,acc_sessao2,crossval_sessao2_media,crossval_sessao2_std,acc_media
0,Sujeito 1,91.49,93.56,4.85,95.74,93.55,4.08,93.62
1,Sujeito 2,95.74,84.14,9.78,93.62,91.85,5.09,94.68
2,Sujeito 3,72.34,69.07,11.90,95.74,91.40,6.01,84.04
3,Sujeito 4,100.00,97.42,0.84,85.11,85.30,6.08,92.55
4,Sujeito 5,93.62,93.99,3.14,95.74,92.66,3.79,94.68
5,Sujeito 6,91.49,89.22,3.60,85.11,87.95,6.01,88.30
6,Sujeito 7,89.36,86.63,5.55,97.87,92.27,2.84,93.62
7,Sujeito 8,95.74,87.57,7.95,87.23,85.72,7.38,91.49
8,Sujeito 9,97.87,97.41,3.20,91.49,89.67,1.53,94.68
9,Sujeito 10,85.11,88.37,5.74,95.74,91.37,4.36,90.43


Classificação de 2 canais

In [17]:
inicio = time.time()
channels = [8,14]
data = pd.DataFrame()
for session in [1,2]:
    sub = []
    acc = []
    crossval = []
    crossval_std = []
    for subject in tqdm(range(1,21)):
        classify = DataClassification(path="dados/carac_subject{0}.csv".format(subject))
        column = 's'+str(session)+'_ch'
        rms_ch = ['rms_'+column+str(i) for i in channels]
        var_ch = ['var_'+column+str(i) for i in channels]
        zcr_ch = ['zcr_'+column+str(i) for i in channels]
        columns = rms_ch+var_ch+zcr_ch
        classify.lda_train(columns)
        acurate = [classify.lda_predict_acc()]
        cval, std_cval = classify.lda_cross_validation()
        acc.extend(acurate)
        crossval.extend([cval])
        crossval_std.extend([std_cval])
        sub.extend(['Sujeito '+str(subject)])
    data['rotulo'] = sub
    data['acc_sessao'+str(session)] = acc
    data['crossval_sessao'+str(session)+'_media'] = crossval
    data['crossval_sessao'+str(session)+'_std'] = crossval_std
data['acc_media'] = (data['acc_sessao1']+data['acc_sessao2'])/2
data = data.append({'rotulo': 'Média', 'acc_sessao1':data.acc_sessao1.mean(), 'acc_sessao2':data.acc_sessao2.mean(),
'crossval_sessao1_media':data.crossval_sessao1_media.mean(), 'crossval_sessao2_media':data.crossval_sessao2_media.mean(),
'crossval_sessao1_std':data.crossval_sessao1_std.mean(), 'crossval_sessao2_std':data.crossval_sessao2_std.mean(), 'acc_media':data.acc_media.mean()
}, ignore_index = True)
data = data.round(2)
fim = time.time()
print("Tempo de execucao: "+str(fim-inicio))
#data.to_csv("dados/acc_score_2ch.csv", sep=';', index = False)

100%|██████████| 20/20 [00:00<00:00, 53.97it/s]

Tempo de execucao: 0.7904713153839111



C:\Users\Lázaro\AppData\Local\Temp\ipykernel_9464\95727770.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [11]:
data

,rotulo,acc_sessao1,crossval_sessao1_media,crossval_sessao1_std,acc_sessao2,crossval_sessao2_media,crossval_sessao2_std,acc_media
0,Sujeito 1,59.57,73.79,9.97,72.34,78.02,10.91,65.96
1,Sujeito 2,72.34,69.85,3.36,78.72,79.79,6.19,75.53
2,Sujeito 3,68.09,62.58,10.41,61.70,61.22,5.88,64.89
3,Sujeito 4,91.49,93.97,2.51,87.23,84.02,4.94,89.36
4,Sujeito 5,85.11,77.17,5.13,87.23,82.39,6.55,86.17
5,Sujeito 6,68.09,61.63,5.43,48.94,58.22,9.77,58.51
6,Sujeito 7,74.47,81.05,5.76,87.23,85.34,4.22,80.85
7,Sujeito 8,80.85,72.51,11.18,59.57,68.97,3.95,70.21
8,Sujeito 9,82.98,75.45,3.88,68.09,65.94,5.08,75.53
9,Sujeito 10,72.34,68.56,8.61,59.57,70.30,9.27,65.96


In [29]:
inicio = time.time()
classify.lda.predict([classify.x[0]])
fim = time.time()
print("Tempo de execucao: "+str(fim-inicio))

Tempo de execucao: 0.000997304916381836
